In [1]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

In [2]:
def loc_shape0(img0, img1, img2, s=128):
    # 가로모임 민글자
    l00 = int(s/2-img0.size[0])
    l01 = int(s/2-img0.size[1]/2)
    l10 = int(s/2)
    l11 = int(s/2-img1.size[1]/2)
    return (l00,l01), (l10,l11), None

def loc_shape1(img0, img1, img2, s=128):
    #세로모임 민글자
    l00 = int(128/2-img0.size[0]/2)
    l01 = int(128/2-img0.size[1])
    l10 = int(128/2-img1.size[0]/2)
    l11 = int(128/2)
    return (l00,l01), (l10,l11), None

def loc_shape2(img0, img1, img2, s=128):
    #섞임모임 민글자
    l00 = int(128/2-img0.size[0])
    l01 = int(128/2-img0.size[1])
    l10 = int(128/2-img1.size[0]/2)
    l11 = int(128/2)
    return (l00,l01), (l10,l11), None

def loc_shape3(img0, img1, img2, s=128):
    #가로모임 받침글자
    l00 = int(128/2-img0.size[0])
    l01 = int(128/2-img0.size[1])
    l10 = int(128/2)
    l11 = int(128/2-img1.size[1])
    l20 = int(128/2-img2.size[0]/2)
    l21 = int(128/2)
    return (l00,l01), (l10,l11), (l20, l21)

def loc_shape4(img0, img1, img2, s=128):
    #세로모임 받침글자
    l00 = int(128/2-img0.size[0]/2)
    l01 = int(128/2-img0.size[1]-img1.size[1]/2)
    l10 = int(128/2-img1.size[0]/2)
    l11 = int(128/2-img1.size[1]/2)
    l20 = int(128/2-img2.size[0]/2)
    l21 = int(128/2+img1.size[1]/2)
    return (l00,l01), (l10,l11), (l20, l21)

def loc_shape5(img0, img1, img2, s=128):
    #섞임모임 받침글자
    l00 = int(128/2-img0.size[0])
    l01 = int(128/2-img0.size[1]-img1.size[1]/2)
    l10 = int(128/2-img1.size[0]/2)
    l11 = int(128/2-img1.size[1]/2)
    l20 = int(128/2-img2.size[0]/2)
    l21 = int(128/2+img1.size[1]/2)
    return (l00,l01), (l10,l11), (l20, l21)


In [3]:
def prepare_data(letter, img0, img1, img2):
    hlist = [0,1,2,3,4,5,6,7,20] # ㅏㅐㅑㅒㅓㅔㅕㅖㅣ
    vlist = [8,12,13,17,18] # ㅗㅛㅜㅠㅡ
    clist = [9,10,11,14,15,16,19] # ㅘㅙㅚㅝㅞㅟㅢ
    
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2
    
    if (ch2 in hlist) & (ch3 == 0):
        # 가로모임 민글자
        wh0, wh1, wh2 = loc_shape0(img0, img1, img2)
        
    elif (ch2 in vlist) & (ch3 == 0):
        # 세로모임 민글자
        wh0, wh1, wh2 = loc_shape1(img0, img1, img2)

    elif (ch2 in clist) & (ch3 == 0):
        # 섞임모임 민글자
        wh0, wh1, wh2 = loc_shape2(img0, img1, img2)

    elif (ch2 in hlist) & (ch3 != 0):
        # 가로모임 받침글자
        wh0, wh1, wh2 = loc_shape3(img0, img1, img2)

    elif (ch2 in vlist) & (ch3 != 0):
        # 세로모임 받침글자
        newsize = (img0.size[0],int(img0.size[1]/3*2))
        img0 = img0.resize(newsize, Image.BILINEAR)
        newsize = (img1.size[0],int(img1.size[1]/3*2))
        img1 = img1.resize(newsize, Image.BILINEAR)
        wh0, wh1, wh2 = loc_shape4(img0, img1, img2)

    elif (ch2 in clist) & (ch3 != 0):
        # 섞임모임 받침글자
        wh0, wh1, wh2 = loc_shape5(img0, img1, img2)

    img = Image.new("L", (128, 128), 255)
    img.paste(img0, wh0)
    img.paste(img1, wh1)
    if wh2 is not None:
        img.paste(img2, wh2)
    
    label = np.zeros([4,128,128])
    label[0,:,:] = 1
    
    npwheres = np.where(np.array(img0)!=255)
    npwheres = (npwheres[0] + wh0[1], npwheres[1] + wh0[0],)
    label[0][npwheres] = 0
    label[1][npwheres] = 1
    
    npwheres = np.where(np.array(img1)!=255)
    npwheres = (npwheres[0] + wh1[1], npwheres[1] + wh1[0],)
    label[0][npwheres] = 0
    label[2][npwheres] = 1
    
    if wh2 is not None:
        npwheres = np.where(np.array(img2)!=255)
        npwheres = (npwheres[0] + wh2[1], npwheres[1] + wh2[0],)
        label[0][npwheres] = 0
        label[3][npwheres] = 1

    return img, label


def get_imgs(fd, font, letter):
    
    ch1_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    ch2_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    ch3_list = [None, 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2

    return (
        Image.open(f"{fd}/{font}/{font}__{ch1_list[ch1]}.png"),
        Image.open(f"{fd}/{font}/{font}__{ch2_list[ch2]}.png"),
        None if ch3_list[ch3] is None else Image.open(f"{fd}/{font}/{font}__{ch3_list[ch3]}.png"),
    )


In [4]:
fd = "/home/jupyter/ai_font/data/exp0717/train_parts"
fonts = os.listdir(fd)
ak = '''가 깨 냐 댸 떠 레 며 볘 삐 사 쌔 야 쟤 쩌 체 켜 톄 피 하 교 꾸 뉴 드 또 료 무 뷰 쁘 소 쑈 우 쥬 쯔 초 쿄 투 
퓨 흐 귀 끠 놔 돼 뙤 뤄 뭬 뷔 쁴 솨 쐐 외 줘 쮀 취 킈 톼 퐤 회 걕 꺾 넧 뎐 뗹 릲 몯 뵬 뿕 슒 쓻 왌 좵 쬞 춿 퀨 튑 픲 핫 갰 꺙 넂 덫 뗔 렽 몦 빟'''
ak = ak.replace("\n","").split(" ")

In [5]:
savefd_assembled = "/home/jupyter/ai_font/data/exp0717/train_seg/raw_assembled"
savefd_label = "/home/jupyter/ai_font/data/exp0717/train_seg/raw_label"

In [6]:
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("to")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

In [7]:
pbar = tqdm(fonts)
pass_count = 0
for font in pbar:
    os.makedirs(f"{savefd_assembled}/{font}",exist_ok=True)
    os.makedirs(f"{savefd_label}/{font}",exist_ok=True)
    for letter in ak:
        try:
            with time_limit(10):
                img0, img1, img2 = get_imgs(fd, font, letter)
                img, label = prepare_data(letter, img0, img1, img2)
                img.save(f"{savefd_assembled}/{font}/{font}__{letter}.png")
                np.save(f"{savefd_label}/{font}/{font}__{letter}.npy", label) 
        except:
            pass_count += 1
    pbar.set_postfix(font=font, pass_count=str(pass_count))

100%|██████████| 426/426 [01:01<00:00,  6.88it/s, font=어비링링체볼드, pass_count=818]     ]          
